In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
#passing the docs and question to the language model to get answers
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_1754/2729103167.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_1754/2729103167.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [4]:
print(vectordb._collection.count())

544


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [ ]:
from openai import OpenAI  #initializing the language model
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)  #lower temp= model is determininstic, it give us more factual asnweres 

/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_1754/2424617482.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0)


In [ ]:
!pip list

## RetreivalQA chain

In [ ]:
from langchain.chains import RetrievalQA  #question answering packed by a retreival style

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm, #the model
    retriever=vectordb.as_retriever() #the vector database
)

In [13]:
result = qa_chain({"query": question})

/var/folders/wp/g4_x_xzn5dj59z7qh4r7bmr40000gn/T/ipykernel_1754/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [14]:
result["result"]

'The major topics covered in this class include OpenCV Multimedia Processing, Object Detection and Tracking with Ultralytics YOLO, Deep Learning concepts such as Neural Networks, CNN architectures, and Transfer Learning, as well as Natural Language Processing topics like Embeddings, RAG Architecture, and Generation Models. Each of these areas involves specific subtopics and techniques that are explored in detail throughout the course.'

## Prompot

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [21]:
question = "Is YOLOv11 a class topic?"

In [22]:
result = qa_chain({"query": question})

In [23]:
result["result"]

'Yes, YOLOv11 is a class topic. Thanks for asking!'

In [24]:
result["source_documents"][0]

Document(metadata={'author': 'Reem Alamoudi', 'creationdate': '2025-02-26T10:46:13+00:00', 'creator': 'Microsoft Word', 'moddate': '2025-02-26T10:46:13+00:00', 'page': 0, 'page_label': '1', 'producer': 'PyPDF', 'source': '/Users/reem/Desktop/Langchain_tutorials/covered topics.pdf', 'total_pages': 6}, page_content='Covered Topics \n• OpenCV Multimedia Processing: \n- Image Filtering Basics \n- Morphological Image processing \n- BITWISWE  \n- Transformation Functions \n- Feature Extraction \n- Operations \n- Blurring Techniques \n- Color Spaces \n- Contour Detection \n- Edge Detection \n- Masking \n- Threshold Techniques \n- Face Detection Using haar cascade \nReference: \nhttps://drive.google.com/file/d/14BsnVqC2c6NNsV4MrAg79ltP_h6Fzd5e/vi\new?usp=share_link \n \n• Object Detection And Tracking With Ultralytics YOLO: \n- Model’s Versions And Capabilities Exploration \n- Bounding Boxes Basics(IOU, mAP, Nms, Confidence Threshold) \n- Understanding Model’s Functionalities')

### RetrievalQA chain types

In [25]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [26]:
result = qa_chain_mr({"query": question})

In [27]:
result["result"]

'No, the text provided does not mention YOLOv11 as a class topic.'

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [36]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "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" # ket from langsmith

In [37]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'The prerequisites, such as pre-training and fine-tuning, are needed to help a model gain general knowledge of instructions and language understanding, enabling zero-shot learning and the ability to generalize to new problems. Fine-tuning is then necessary to adapt the pre-trained model to specific downstream tasks, ensuring effective learning and performance across various tasks simultaneously.'

In [35]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'The prerequisites mentioned in the context are needed to build a foundation for understanding pre-training in the context of AI. These prerequisites help in grasping the basics of small-sized data, complex reasoning, and planning abilities, which are essential for effective pre-training techniques. Understanding these concepts is crucial for further exploring and implementing pre-training methods in artificial intelligence, including choosing appropriate training objectives through careful experimentation and categorizing pre-training tasks based on training objectives such as language modeling and masked language modeling. This understanding is particularly important for tasks like writing and code generation, where complex reasoning and compositional thinking are required, and where the reasoning steps may not be fixed, varying for different problems with each step depending on the outcomes of prior steps. Additionally, having a strong foundation in language understanding and genera

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [38]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [40]:
question = "Is Opencv a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, OpenCV is a class topic covered in the context provided. It includes various aspects of OpenCV multimedia processing such as image filtering basics, morphological image processing, color spaces, contour detection, edge detection, masking, threshold techniques, and more.'

In [41]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

"I don't have enough context to provide a specific answer to your question about why those prerequisites are needed."

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.